<a href="https://colab.research.google.com/github/Kenjiro1/Formacao_IA/blob/main/NLP/NLPRna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Kenjiro1/Formacao_IA

Cloning into 'Formacao_IA'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 210 (delta 4), reused 0 (delta 0), pack-reused 196 (from 1)
Receiving objects: 100% (210/210), 35.94 MiB | 30.59 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [20]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

In [3]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

In [4]:
spam = pd.read_csv('/content/Formacao_IA/dados/spam.csv')
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
print(spam.shape,': Tamanho')

count = spam['Category'].value_counts()
print(count)

# Temos muito mais exemplares de ham do que spam


(5572, 2) : Tamanho
Category
ham     4825
spam     747
Name: count, dtype: int64


In [6]:
ham_samples = spam[spam['Category'] == 'ham'].sample(n = 747, random_state=42)
spam_samples = spam[spam['Category'] == 'spam']

spam = pd.concat([ham_samples,spam_samples]).sample(frac=1, random_state=42).reset_index(drop=True)


In [7]:
spam.shape #Balanceados

(1494, 2)

In [16]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(spam['Category'])
print(y)

mensagens = spam['Message'].values
print(mensagens)

X_train,X_test, Y_train, Y_test = train_test_split(mensagens, y, test_size=0.3,random_state= 43)

token = Tokenizer(num_words=1000, lower=False)
token.fit_on_texts(X_train)
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

[1 1 1 ... 1 1 1]
['URGENT, IMPORTANT INFORMATION FOR O2 USER. TODAY IS YOUR LUCKY DAY! 2 FIND OUT WHY LOG ONTO HTTP://WWW.URAWINNER.COM THERE IS A FANTASTIC SURPRISE AWAITING FOR YOU'
 'Panasonic & BluetoothHdset FREE. Nokia FREE. Motorola FREE & DoubleMins & DoubleTxt on Orange contract. Call MobileUpd8 on 08000839402 or call 2optout'
 'Do you want a new Video handset? 750 any time any network mins? UNLIMITED TEXT? Camcorder? Reply or Call now 08000930705 for del Sat AM'
 ...
 'You have an important customer service announcement from PREMIER. Call FREEPHONE 0800 542 0578 now!'
 'URGENT! Your Mobile number has been awarded with a £2000 prize GUARANTEED. Call 09061790126 from land line. Claim 3030. Valid 12hrs only 150ppm'
 'How come it takes so little time for a child who is afraid of the dark to become a teenager who wants to stay out all night?']


In [19]:
print(X_test)

#Iremos padronizar agora os nossos textos em um tamanho padrao

X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test= pad_sequences(X_test, padding='post', maxlen=500)
X_test

[[396 201 939 ...   0   0   0]
 [172 134  11 ...   0   0   0]
 [ 72   3  51 ...   0   0   0]
 ...
 [615  15   2 ...   0   0   0]
 [265 146   2 ...   0   0   0]
 [  6   6  24 ...   0   0   0]]


array([[396, 201, 939, ...,   0,   0,   0],
       [172, 134,  11, ...,   0,   0,   0],
       [ 72,   3,  51, ...,   0,   0,   0],
       ...,
       [615,  15,   2, ...,   0,   0,   0],
       [265, 146,   2, ...,   0,   0,   0],
       [  6,   6,  24, ...,   0,   0,   0]], dtype=int32)

In [31]:
#Iremos criar a rede neural
input_layer = Input(shape=(500,))
embedding_layer = Embedding(input_dim=len(token.word_index), output_dim=50)(input_layer)
flatten_layer = Flatten()(embedding_layer)
dense_layer = Dense(units=10, activation='relu')(flatten_layer)
dropout_layer = Dropout(0.1)(dense_layer)
output_layer = Dense(units=1, activation='sigmoid')(dropout_layer)
modelo = Model(inputs=input_layer, outputs=output_layer)

modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])
modelo.fit(X_train, Y_train, epochs=20, batch_size=10, verbose=True, validation_data=(X_test,Y_test))

Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5724 - loss: 0.2340 - val_accuracy: 0.8976 - val_loss: 0.0964
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8753 - loss: 0.0857 - val_accuracy: 0.9599 - val_loss: 0.0484
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9261 - loss: 0.0494 - val_accuracy: 0.9644 - val_loss: 0.0405
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9622 - loss: 0.0374 - val_accuracy: 0.9666 - val_loss: 0.0321
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9832 - loss: 0.0204 - val_accuracy: 0.9666 - val_loss: 0.0345
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9805 - loss: 0.0219 - val_accuracy: 0.9666 - val_loss: 0.0347
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9817 - loss: 0.0182 - val_accuracy: 0.9666 - val_loss: 0.0275
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9856 - loss: 0.0170 - val_accuracy: 0.

In [32]:
loss, accuracy = modelo.evaluate(X_test, Y_test)
print('loss = ', loss)
print('accuracy = ', accuracy)

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9539 - loss: 0.0377 
loss =  0.033257488161325455
accuracy =  0.9576837420463562


Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5144 - loss: 0.2499 - val_accuracy: 0.4833 - val_loss: 0.2503
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5144 - loss: 0.2499 - val_accuracy: 0.4833 - val_loss: 0.2503
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5144 - loss: 0.2499 - val_accuracy: 0.4833 - val_loss: 0.2503
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5144 - loss: 0.2499 - val_accuracy: 0.4833 - val_loss: 0.2503
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5144 - loss: 0.2499 - val_accuracy: 0.4833 - val_loss: 0.2503
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5144 - loss: 0.2499 - val_accuracy: 0.4833 - val_loss: 0.2503
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5144 - loss: 0.2499 - val_accuracy: 0.4833 - val_loss: 0.2503
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5144 - loss: 0.2499 - val_accuracy: 